In [1]:
import math
import torch
import gpytorch
import numpy as np
from matplotlib import pyplot as plt
import test_functions

%matplotlib inline
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [2]:
from torch.autograd import Variable
from torch import optim, nn
from gpytorch.kernels import RBFKernel, GridInterpolationKernel
from gpytorch.means import ConstantMean
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.random_variables import GaussianRandomVariable
from scipy.stats import norm
from scipy.optimize import minimize

In [3]:
class Exact_RBF_GP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, bounds):
        super(Exact_RBF_GP, self).__init__(train_x, train_y, likelihood)
        # Our mean function is constant in the interval [-1,1]
        self.mean_module = ConstantMean(constant_bounds=(-1, 1))
        # We use the RBF kernel as a universal approximator
        self.covar_module = RBFKernel(log_lengthscale_bounds=(-5, 5))
    
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        # Return moddl output as GaussianRandomVariable
        return GaussianRandomVariable(mean_x, covar_x)

In [4]:
class Interp_RBF_GP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, bounds):
        super(Interp_RBF_GP, self).__init__(train_x, train_y, likelihood)
        d = len(bounds)
        # Near-zero mean
        self.mean_module = ConstantMean(constant_bounds=[-1e-5,1e-5])
        # GridInterpolationKernel over an ExactGP
        self.base_covar_module = RBFKernel(log_lengthscale_bounds=(-5, 6))
        self.covar_module = GridInterpolationKernel(self.base_covar_module, grid_size=60//d,
                                                    grid_bounds=bounds)
        # Register the log lengthscale as a trainable parametre
        self.register_parameter('log_outputscale', nn.Parameter(torch.Tensor([0])), bounds=(-5,6))
        
    def forward(self,x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        covar_x = covar_x.mul(self.log_outputscale.exp())
        return GaussianRandomVariable(mean_x, covar_x)

In [5]:
class EI_Acq(torch.nn.Module):
    
    def __init__(self, gp, y_max, xi, bounds):
        super(EI_Acq, self).__init__()
        dim = len(bounds)
        
        sample = torch.Tensor(1, 1).uniform_(bounds[0][0], bounds[0][1])
        for i in range(1, len(bounds)):
            sample = torch.cat([sample, torch.Tensor(1, 1).uniform_(bounds[i][0], bounds[i][1])], dim=1)
        
        self.x = nn.Parameter(sample)
        self.gp = gp
        self.xi = xi
        self.y_max = y_max
        
    def forward(self):
        with gpytorch.fast_pred_var():
            predict_pt = self.gp(self.x)
        mean = predict_pt.mean()
        std = torch.sqrt(predict_pt.var())
        z = (mean - self.y_max - self.xi)/std
        return (
            (mean - self.y_max - self.xi) * Variable(torch.from_numpy(norm.cdf(z.data)).float()) + 
            std * Variable(torch.from_numpy(norm.pdf(z.data)).float())
            ) 
        
        return self.x

In [6]:
class Bayes_opt:
    
    def __init__(self, func, bounds, step_num):
        
        self.dim = len(bounds)
        self.bounds = bounds
        self.step_num = step_num    
        self.obj_func = func
        self.xi = 0.01
        self.model_GP = None
        
        #Allocate space for iterates, initialize algorithm
        start_num = 2
        sample = torch.Tensor(start_num, self.dim)
        for i in range(self.dim):
            sample[:, i] = torch.Tensor(start_num, 1).uniform_(bounds[i][0], bounds[i][1])
        
        self.sample_pts = Variable(torch.zeros(self.step_num, self.dim))     
        self.sample_vals = Variable(torch.zeros(self.step_num))
        start_pts = Variable(sample)
        start_vals = self.obj_func(start_pts)
        self.sample_pts[:start_num] = start_pts
        self.sample_vals[:start_num] = start_vals
        
        ind = torch.min(start_vals.data, 0)[1]
        self.opt_val = torch.min(start_vals.data)
        self.opt_soln = start_pts[ind]
        self.model_GP = self._update_GP(start_pts, start_vals)
        print("GP initiated")
        
    
    def _acq_max(self):
        
        tol = 10^-1
        n_repeats = 10

        for rep in range(n_repeats):

            f = EI_Acq(self.model_GP, self.opt_val, self.xi, self.bounds)
            optimizer = torch.optim.Adam([{'params': f.x}], lr=0.01/self.dim)
            prev_loss = 0
            bound_flag = True

            for t in range(50):
                optimizer.zero_grad()
                loss = -f()
                loss.backward(retain_graph=True)
                optimizer.step()

                if ((loss-prev_loss)**2 < tol).data.numpy(): break
                prev_loss = loss

            for i in range(self.dim):
                if (f.x[:, i] < self.bounds[i][0]).any() or (f.x[:, i] > self.bounds[i][1]).any():
                    bound_flag = False
                    break
                
            if bound_flag == True:
                break
                
                
        clamped_pt = torch.Tensor(1, self.dim)
        for i in range(self.dim):
            clamped_pt[:, i] = torch.clamp(f.x.data[:, i], self.bounds[i][0], self.bounds[i][1])
        
        return Variable(clamped_pt)
    
    def _update_GP(self, train_x, train_y, train_hyperparams=False):
        
        likelihood = GaussianLikelihood()
        model = Exact_RBF_GP(train_x.data, train_y.data, likelihood, self.bounds)
        
        if not self.model_GP is None:
            model.parameters = self.model_GP.parameters

        if train_hyperparams:
            model.train()
            likelihood.train()
            # Use the adam optimizer
            optimizer = torch.optim.Adam([
                {'params': model.parameters()},  # Includes GaussianLikelihood parameters
            ], lr=0.1)

            # "Loss" for GPs - the marginal log likelihood
            mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

            training_iter = 50
            for i in range(training_iter):
                # Zero gradients from previous iteration
                optimizer.zero_grad()
                # Output from model
                output = model(train_x)
                # Calc loss and backprop gradients
                loss = -mll(output, train_y)
                loss.backward()
                optimizer.step()
            
    
        # Put model and likelihood into eval mode
        model.eval()
        likelihood.eval()

        return model
        
        
    def optimize(self, mode="compact"):
        
        start_num = int(self.sample_pts.nonzero().size(0)/self.dim)
        if mode=="eval":
            self.obj_hist = Variable(torch.zeros(step_num))
            self.obj_hist[:start_num] = self.opt_val
        
        for t in range(start_num, self.step_num):
            
            #Get next point from acquisition function, evaluate objective
            new_pt = self._acq_max()
            new_val = float(self.obj_func(new_pt))
            self.sample_pts[t] = new_pt
            self.sample_vals[t] = new_val
            
            if new_val > self.opt_val:
                self.opt_val = new_val
                self.opt_soln = new_pt
                
            if mode=="eval":
                self.obj_hist[t] = self.opt_val
                
            #Update GP with new observation
            if t%25 == 0: 
                #print(t, ': ', self.opt_val)
                self.model_GP = self._update_GP(self.sample_pts[:t+1], self.sample_vals[:t+1], train_hyperparams=True)
            else:
                self.model_GP = self._update_GP(self.sample_pts[:t+1], self.sample_vals[:t+1])

In [7]:
class Ackley_SGD(torch.nn.Module):
    #Global max of 0 at origin
    
    def __init__(self, bounds):
        super(Ackley_SGD, self).__init__()
        self.dim = len(bounds)
        sample = torch.Tensor(1, 1).uniform_(bounds[0][0], bounds[0][1])
        for i in range(1, self.dim):
            sample = torch.cat([sample, torch.Tensor(1, 1).uniform_(bounds[i][0], bounds[i][1])], dim=1)

        self.x = nn.Parameter(sample)
        
    def forward(self):
        a = 20; b = 0.2; c=2*math.pi;

        term_1 = -a * torch.exp(-b*torch.sqrt(1/self.dim*torch.sum(self.x**2, dim=1)))
        term_2 = -1*torch.exp(1/self.dim*torch.sum(torch.cos(c*self.x), dim=1))

        return -1*(term_1 + term_2 + a + math.exp(1))
    
def Adam(f, bounds):
    
    tol = 10^-1
    n_repeats = 10
    dim = len(bounds)
    opt_pt = Variable(torch.zeros(n_repeats, dim))
    opt_val = Variable(torch.zeros(n_repeats))

    for rep in range(n_repeats):

        optimizer = torch.optim.Adam([{'params': f.x}], lr=0.01/dim)
        prev_loss = 0
        bound_flag = True

        for t in range(10**4):
            optimizer.zero_grad()
            loss = -f()
            loss.backward(retain_graph=True)
            optimizer.step()

            if ((loss-prev_loss)**2 < tol).data.numpy(): break
            prev_loss = loss

        for i in range(dim):
            if (f.x[:, i] < bounds[i][0]).any() or (f.x[:, i] > bounds[i][1]).any():
                bound_flag = False
                break

        if bound_flag == True:
            opt_val[rep] = f()
            opt_pt[rep, :] = f.x
            
    return opt_pt, opt_val

In [8]:
bounds = [(-2, 2)]*3
n_trials = 5

In [9]:
%%time

result_adam = Variable(torch.Tensor(n_trials))
for t in range(n_trials):
    f = Ackley_SGD(bounds)
    opt_pt, opt_val = Adam(f, bounds)
    result_adam[t] = opt_val[(opt_val <= 0).data].max()
    
print(result_adam.mean())

Variable containing:
-3.3901
[torch.FloatTensor of size 1]

CPU times: user 1min 11s, sys: 5.51 s, total: 1min 17s
Wall time: 1min 20s


In [10]:
%%time

step_num = 100
result_bayesopt = Variable(torch.Tensor(n_trials))
for t in range(n_trials):
    try:
        test = Bayes_opt(func=test_functions.ackley, bounds=bounds, step_num=step_num)
        test.optimize(mode="eval")
        result_bayesopt[t] = test.opt_val
    except:
        continue

#plt.plot(range(step_num), test.obj_hist.data.numpy())

print(result_bayesopt.mean())

GP initiated
GP initiated
GP initiated
GP initiated
GP initiated
Variable containing:
-1.3952
[torch.FloatTensor of size 1]

CPU times: user 1min 16s, sys: 464 ms, total: 1min 16s
Wall time: 1min 25s


In [11]:
print("Adam Results: Mean Optimal Value = {}, Variance = {}".format(result_adam.mean().data[0], result_adam.var().data[0]))
print("BayesOpt Results: Mean Optimal Value = {}, Variance = {}".format(result_bayesopt.mean().data[0], result_bayesopt.var().data[0]))

Adam Results: Mean Optimal Value = -3.390089988708496, Variance = 4.63358736038208
BayesOpt Results: Mean Optimal Value = -1.3951585292816162, Variance = 0.2216348499059677


In [14]:
result_bayesopt

Variable containing:
-1.1398
-2.1999
-1.3294
-0.9914
-1.3152
[torch.FloatTensor of size 5]